In [0]:
import numpy as np
import csv
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

np.random.seed(113)
path_ukara = "drive/My Drive/Ukara/"
path_nlp = "drive/My Drive/a/"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_A = pd.read_csv(path_ukara + "data_train_A.csv").values
data_B = pd.read_csv(path_ukara + "data_train_B.csv").values

In [0]:
data_A[:3]

array([['TRA1', 'intetraksi/beradaptasi terhadap lingkungan yang baru.',
        1],
       ['TRA2', 'seperti jatuhnya meteor tsunami gempa bumi', 0],
       ['TRA3', 'hanya tuhan yang tahu tantangan nya itu apaan', 0]],
      dtype=object)

In [0]:
xtrain_A, xtest_A, ytrain_A, ytest_A = train_test_split(data_A[:,:-1], data_A[:,-1],
                                      test_size=0.2, 
                                      stratify=data_A[:,-1], 
                                      random_state=113)

xtrain_B, xtest_B, ytrain_B, ytest_B = train_test_split(data_B[:,:-1], data_B[:,-1],
                                      test_size=0.2, 
                                      stratify=data_B[:,-1], 
                                      random_state=113)

In [0]:
skf = StratifiedKFold(n_splits=5, random_state=113, shuffle=False)

In [0]:
print(xtrain_A[0], ytrain_A[0])

['TRA226' 'negara munggin menjari donasi hidup baru'] 0


In [0]:
print(np.unique(data_A[:,-1], return_counts=True))
print(np.unique(data_B[:,-1], return_counts=True))

(array([0, 1], dtype=object), array([ 77, 191]))
(array([0, 1], dtype=object), array([137, 168]))


In [0]:
def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

# Fasttext Part

In [0]:
!pip install fasttext

     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2384810 sha256=8f8f730e115254945e09ef2636fed4dd2418c1e43ebefaf0de492c0d8c833f77
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz

--2019-10-06 15:08:31--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4507049071 (4.2G) [application/octet-stream]
Saving to: ‘cc.id.300.bin.gz’

cc.id.300.bin.gz    100%[===================>]   4.20G  29.6MB/s    in 2m 28s  

2019-10-06 15:11:00 (29.0 MB/s) - ‘cc.id.300.bin.gz’ saved [4507049071/4507049071]



In [0]:
!gunzip cc.id.300.bin.gz

In [0]:
# !mv "cc.id.300.bin" "drive/My Drive/a/"

In [0]:
import fasttext
model = fasttext.load_model(path_nlp + "trained_fasttext_ukara.pk")
# print(model['mesin'][:3])

In [0]:
import fasttext
model2 = fasttext.load_model("cc.id.300.bin")

# PreProcess

In [0]:
!pip install PySastrawi

     |████████████████████████████████| 215kB 2.8MB/s 


In [0]:
import pandas as pd
import string
from nltk.util import skipgrams
from nltk import ngrams
from collections import Counter
import math

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [0]:
lena = []
for d in data_A[:,1]:
    lena.append(len(str(d).split()))
datalen_mean_A = np.mean(lena)
datalen_std_A = np.std(lena)

lenb = []
for d in data_B[:,1]:
    lenb.append(len(str(d).split()))
datalen_mean_B = np.mean(lenb)
datalen_std_B = np.std(lenb)

stop_b = ['yang', 'lebih', 'untuk', 'dan']
stop_a = ['yang', 'akan', 'mereka', 'dan']

table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
def rmpun(text):
    return text.translate(table)
    
def preprocess(data, tipe="A"):
    if type(data) != str:
        return " "
    if len(data) <1:
        return " "
    data = data.lower()
    data = data.replace('/',' atau ')
    data = data.replace('yg ','yang ')
    data = rmpun(data)
    data = " ".join(data.split())
    if tipe=="A":
        for w in stop_a:
            data = data.replace(w, '')
    else:
        for w in stop_b:
            data = data.replace(w, '')
    return data

def preprocess_2(data, tipe="A"):
    data = data.lower()
    data = data.replace('/',' atau ')
    data = data.replace('"','')
    data = data.replace('\'','')
    data = data.replace('yg ','yang ')
    data = data.replace('_',' ')
    data = data.replace(',',' , ')
    return data

def vectorize(x, pre=True, tipe="A"):
    ans = []
    for data in x:
        if pre:
            data = preprocess(str(data), tipe)
        vect = model.get_sentence_vector(data)
        ans.append(vect)
    return np.asarray(ans)

def vectorize2(x, pre=True, tipe="A"):
    ans = []
    for data in x:
        if pre:
            data = preprocess_2(str(data), tipe)
        vect = model2.get_sentence_vector(data)
        ans.append(vect)
    return np.asarray(ans)

def find_len(x, tipe='A'):
    ans = []
    for data in x:
        ans.append(len(str(data).split()))
    ans = np.asarray(ans, dtype="float32")
    
    if tipe=="A":
        datalen_mean = datalen_mean_A
        datalen_std = datalen_std_A
    else:
        datalen_mean = datalen_mean_B
        datalen_std = datalen_std_B
    
    ans -= datalen_mean
    return ans / datalen_std

# 1, 58

In [0]:
# table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
# def rmpun(text):
#     return text.translate(table)
    
# def preprocess_istri(data):
#     if type(data) != str:
#         return " "
#     if len(data) <1:
#         return " "
#     data = data.lower()
#     data = data.replace('/',' atau ')
#     data = data.replace('yg ','yang ')
#     data = rmpun(data)
#     data = " ".join(data.split())
#     # data = stem(data)
#     return data

# word_b = ""
# for data in data_A:
#     word_b += preprocess_istri(data[1])
# counter = Counter(word_b.split())
# counter = list(reversed(counter.most_common()))
# counter[:50]

# PPMI

In [0]:
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
def rmpun(text):
    return text.translate(table)

def stem(text):
    text = stemmer.stem(text)
    return text

def preprocess_ppmi(data):
    if type(data) != str:
        return " "
    if len(data) <1:
        return " "
    data = data.lower()
    data = data.replace('/',' atau ')
    data = data.replace('yg ','yang ')
    data = rmpun(data)
    data = " ".join(data.split())
    # data = stem(data)
    return data

def pmi(w1, w2, unifreq, bifreq):
    pw1 = unifreq[w1] / float(sum(unifreq.values()))
    pw2 = unifreq[w2] / float(sum(unifreq.values()))
    pw1w2 = bifreq[" ".join([w1, w2])] / float(sum(bifreq.values()))
    try:
        res = math.log(pw1w2/float(pw1*pw2),2)
        if res < 0:
            res = 0
    except:
        res = 0
    return res

def mkbigram(text):
    gram2 = list(ngrams(text.split(), 2))
    if len(gram2) == 0:
        return [text]
    return gram2

In [0]:
traina1ex = ["Sebuah negara membeli pulau membeli tanah untuk memindahkan penduduknya di sana suatu hari.",
                "Pemerintah berharap tidak menempatkan semua penduduk semua warga semua orang di sebidang tanah.", 
                "Tetapi jika itu diperlukan untuk menyelamatkan warga penduduk maka akan dilakukan.",
                "Pulau tanah itu akan menjadi rumah baru bagi penduduk warga yang berpindah.",
                "Relokasi warga penduduk karena perubahan iklim.",
                "Sebuah negara sudah mulai memindahkan populasi penduduk dari daratan rendah ke sebuah pulau lain.",
                "Tapi ini akan menjadi pertama kalinya seluruh negara harus pindah karena tanah yang dibangun tidak lagi ada.",
                "Ini menimbulkan pertanyaan baru dan menakutkan: Jika suatu negara tidak lagi ada dalam bentuk fisik, dapatkah ia tetap ada sebagai entitas politik?",
                "Bisakah suatu bangsa bangkit dan bergerak?",
                "Jika negara itu berpindah kita tidak tahu apakah kebangsaan bangsa penduduknya bisa dilestarikan.",
                "Negara harus tinggal, dilestarikan, jika memungkinkan.",
                "Relokasi mungkin tidak dapat dihindari.",
                "Meski dengan aktivis gerakan nol emisi tetapi negara itu masih tenggelam.",
                "Secara historis, negara tidak dihancurkan secara fisik; mereka hanya menjadi negara lain,",
                "tanah yang mereka tempati dikendalikan oleh orang lain.",
                "Tetapi setidaknya, untuk eksis, suatu negara membutuhkan pemerintah, populasi, dan sebidang pemukiman di dalam wilayah yang baru.",
                "untuk mempertahankan pengakuan internasional, negara kepulauan yang menghadapi kehancuran karena perubahan iklim harus memperkuat wilayah mereka",
                "menjaga setidaknya beberapa struktur fisik di atas air dan menjaga sekelompok kecil penduduk, bahkan jika sebagian besar penduduk telah pindah.",
                "Orang-orang dari negara kepulauan kecil ingin terus memiliki perwakilan politik dalam komunitas internasional, dan mereka memiliki kepentingan ekonomi untuk dilindungi - hak atas perikanan dan sumber daya alam di wilayah mereka",
                "Sangat masuk akal bahwa dampak perubahan iklim akan membuat negara bangsa tertentu tidak dapat dihuni sebelum akhir abad ini.",
                "Walaupun ini mungkin nasib sejumlah kecil negara bangsa yang paling rentan terhadap perubahan iklim",
                "implikasinya terhadap evolusi kenegaraan dan hukum internasional.",
                "Untuk menanggapi fenomena negara tak bertanah.",
                "status yang memungkinkan keberlangsungan keberadaan negara berdaulat, memberikan semua hak dan manfaat kedaulatan di antara keluarga negara, untuk selamanya.",
                "Dalam praktiknya hal ini membutuhkan pembentukan kerangka kerja pemerintah baru yang dapat menggunakan wewenang atas penduduk yang tersebar karena perpindahan",
                "Prospek pendidikan dan pekerjaan di pulau terbatas, dan mayoritas anak muda yang keluarganya mampu berpindah ke negara lain.",
                "Banyak orang berniat bermigrasi untuk merespons perubahan iklim.",
                "sebagian besar generasi tua tidak ingin pindah karena mereka percaya akan kehilangan identitas, budaya, gaya hidup dan tradisi mereka.",
                "Tapi pemerintah percaya bahwa generasi muda berniat untuk bermigrasi demi keselamatan generasi mendatang."]
len(traina1ex)

29

In [0]:
def get_ppmi(x_lengkap):
    traina1 = x_lengkap[x_lengkap[:,2] == 1, 1]
    # print("===================")
    # print(traina1)
    # print("===================")
    # cleansing
    trainAclear = [preprocess_ppmi(text) for text in list(traina1)]
    trainAclear.extend([preprocess_ppmi(text) for text in list(traina1ex)])

    # combination skipgram
    unigram = []
    bigram = []
    for sent in trainAclear:        
        unigram.extend(sent.split())
        bigram.extend([" ".join(item) for item in list(ngrams(sent.split(), 2))])
        
    count = Counter(unigram)
    countbigram = Counter(bigram)

    # print("===================")
    # print(unigram)
    # print("===================")
    # print(bigram)
    # print("===================")

    # traina['CL RESPONSE'] = traina.RESPONSE.apply(preprocess)
    PPMI = []
    idx = 0
    for sent in x_lengkap[:,1]:
        sent = preprocess_ppmi(sent)
        try:
            big = mkbigram(sent)
            ppmi = sum([pmi(item[0],item[1],count,countbigram) for item in big])/len(big)
            PPMI.append(ppmi)
        except:
            PPMI.append(0)
        # print(idx)
        idx+=1

    return PPMI, count, countbigram

def get_ppmi_test(x_lengkap, count, countbigram):
    PPMI = []
    idx = 0
    for sent in x_lengkap[:,1]:
        sent = preprocess_ppmi(sent)
        try:
            big = mkbigram(sent)
            ppmi = sum([pmi(item[0],item[1],count,countbigram) for item in big])/len(big)
            PPMI.append(ppmi)
        except:
            PPMI.append(0)
        # print(idx)
        idx+=1
    return PPMI

# Classifier Booster 1


In [0]:
!pip install xgboost imblearn

In [0]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


### DATA A

In [0]:
X = vectorize2(data_A[:,1].copy(), pre=True, tipe="A")
# XE = np.load("A_X.npy")
Y = np.load("A_Y.npy")

X = np.c_[X, find_len(data_A[:,1])]

sampler = SMOTE(sampling_strategy=0.5, random_state=113)

# clf = MLPClassifier(
#         random_state=113,
#         max_iter=250,
#         hidden_layer_sizes=(200,), # 250,256-64
#         beta_1=0.9,
#         beta_2=0.999,
#         learning_rate_init=0.0082,
#         learning_rate = 'adaptive',
#         solver='adam')

hiddens = [(100,),(200,),(300,),(400)]
hiddens = [100, 150, 200]
lris = [0.001, 0.01, 0.005]
solvers = ['adam']
max_iters = [300, 400,500]
max_iters = [0.8, 0.9]
# lrs = ['adaptive', 'constant', 'invscaling']
use_ppmi = False

# hiddens = [(250,)]
# lris = [0.0005]
# solvers = ['adam']
# max_iters = [300]
lrs = ['constant']

skf = StratifiedKFold(n_splits=5, random_state=113, shuffle=False)

for h in hiddens:
    for lri in lris:
        for solver in solvers:
            for mi in max_iters:
                for lr in lrs:
                    print()
                    print(h, lri, solver, mi, lr)
                    
                    avg_f1 = 0
#                     clf = MLPClassifier(
#                             random_state=113,
#                             max_iter=mi,
#                             hidden_layer_sizes=h, # 250,256-64
#                             learning_rate_init=lri,
#                             learning_rate = lr,
#                             solver=solver)
                    
                    
                    clf = XGBClassifier(random_state=113, n_estimators=h,
                                        max_depth=3, learning_rate=lri, subsample=mi)
                    
                    done = True
                    for i, (train_index, test_index) in enumerate(skf.split(X, Y)):
                        done = False
                        x_train, y_train = X[train_index], Y[train_index]
                        
                        if use_ppmi:
                            ppmi_x_train, ppmic, ppmicb = get_ppmi(data_A[train_index])
                            x_train = np.c_[x_train, ppmi_x_train]

                        x_train, y_train = sampler.fit_resample(x_train, y_train)

                        x_test, y_test = X[test_index], Y[test_index]
                        
                        if use_ppmi:
                            ppmi_x_test = get_ppmi_test(data_A[test_index], ppmic, ppmicb)
                            x_test = np.c_[x_test, ppmi_x_test]

                        clf.fit(x_train, y_train)

                        prediction = clf.predict(x_test).astype(int)

                        # print("Accuracy pada train data:", accuracy_score(clf.predict(x_train).astype(int), y_train))
                        # print("Accuracy pada test data:", accuracy_score(prediction, y_test))
                        # print("Precision baseline pada test data:", precision_score(prediction, y_test))
                        # print("Recall baseline pada test data:", recall_score(prediction, y_test))
                        print("F1 baseline pada test data:", f1_score(prediction, y_test))
                        print()
                        avg_f1 += f1_score(prediction, y_test)

#                         if f1_score(prediction, y_test)<0.9 and i==0:
#                             break
#                         if f1_score(prediction, y_test)<0.87 and i==3:
#                             break
                        done = True
                    
                    if done:
                        print("Average F1", avg_f1/5)

                    # clf = MLPClassifier(
                    #         random_state=113,
                    #         max_iter=mi,
                    #         hidden_layer_sizes=h,
                    #         learning_rate_init=lri,
                    #         learning_rate = lr,
                    #         solver=solver)
                    
                    # if use_ppmi:
                    #     X = np.c_[X, ppmi_a]

                    # X_resampled, Y_resampled = sampler.fit_resample(X, Y)
                    # clf.fit(X_resampled, Y_resampled)
   


100 0.001 adam 0.8 constant
F1 baseline pada test data: 0.8604651162790696

F1 baseline pada test data: 0.7894736842105263

F1 baseline pada test data: 0.8333333333333333

F1 baseline pada test data: 0.8275862068965517

F1 baseline pada test data: 0.8860759493670887

Average F1 0.8393868580173139

100 0.001 adam 0.9 constant
F1 baseline pada test data: 0.8604651162790696

F1 baseline pada test data: 0.7894736842105263

F1 baseline pada test data: 0.8192771084337349

F1 baseline pada test data: 0.8470588235294116

F1 baseline pada test data: 0.9090909090909091

Average F1 0.8450731283087304

100 0.01 adam 0.8 constant
F1 baseline pada test data: 0.8705882352941177

F1 baseline pada test data: 0.810126582278481

F1 baseline pada test data: 0.8333333333333333

F1 baseline pada test data: 0.8505747126436782

F1 baseline pada test data: 0.9367088607594938

Average F1 0.8602663448618209

100 0.01 adam 0.9 constant
F1 baseline pada test data: 0.8470588235294118

F1 baseline pada test data: 0

In [0]:
# STACKING LV 0 DATA A
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

avg_f1 = 0

# ACTIVATE FOR EXPLORING
for p1 in [(250,)]: #  hiddens
    for p2 in [0.0001]: # learning_rate
        for p3 in [800]: # max_iter
# for p1 in [75]: # n_estimator, hiddens
#     for p2 in [ 0.001]: # learning_rate
#         for p3 in [0.75]: # subsample, max_iter
            avg_exp_f1 = 0
            print()
            print(p1, p2, p3)

            np.random.seed(113)
            sampler = SMOTE(sampling_strategy=0.5, random_state=113)

            X = data_A[:,1].copy()
            Y = np.load("A_Y.npy")
            skf = StratifiedKFold(n_splits=5, random_state=113, shuffle=False)

            for kf, (id_train, id_test) in enumerate(skf.split(X, Y)):
                print("Fold %d/5" % (kf+1))

                X_train = vectorize(data_A[id_train,1].copy(), pre=True)
                Y_train = np.load("A_Y.npy")[id_train]

                X_test = vectorize(data_A[id_test,1].copy(), pre=True)
                Y_test = np.load("A_Y.npy")[id_test]

                X_test = np.c_[X_test, find_len(data_A[id_test,1], tipe='A')]
                X_train = np.c_[X_train, find_len(data_A[id_train,1], tipe='A')]

                X_train, Y_train = sampler.fit_resample(X_train, Y_train)

                # XGB
                # clf_b_1 = XGBClassifier(random_state=113, n_estimators=p1, learning_rate=p2, subsample=p3)
                # clf_b_1.fit(X_train, Y_train)

                # # MLP 
                clf_b_2 = MLPClassifier(random_state=113, max_iter=p3, hidden_layer_sizes=p1, learning_rate_init=p2, solver='adam')
                clf_b_2.fit(X_train, Y_train)

                # ev1 = clf_b_1.predict(X_test)
                ev2 = clf_b_2.predict(X_test)

                avg_exp_f1 += f1_score(ev2, Y_test) # <<<< change this

                # print("M1 Accuracy pada test data:", accuracy_score(ev1, Y_test))
                # print("   F1 pada test data:", f1_score(ev1, Y_test))
                print("M2 Accuracy pada test data:", accuracy_score(ev2, Y_test))
                print("   F1 pada test data:", f1_score(ev2, Y_test))

            print("AVERAGE F1", avg_exp_f1/5)


(250,) 0.0001 800
Fold 1/5


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M2 Accuracy pada test data: 0.8727272727272727
   F1 pada test data: 0.9135802469135802
Fold 2/5


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M2 Accuracy pada test data: 0.7592592592592593
   F1 pada test data: 0.8354430379746836
Fold 3/5


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M2 Accuracy pada test data: 0.7358490566037735
   F1 pada test data: 0.8292682926829269
Fold 4/5


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M2 Accuracy pada test data: 0.8867924528301887
   F1 pada test data: 0.9268292682926829
Fold 5/5
M2 Accuracy pada test data: 0.9245283018867925
   F1 pada test data: 0.9444444444444444
AVERAGE F1 0.8899130580616637


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
# STACKING LV 1 DATA A
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

for p1 in [(45,)]: # n_estimator, hiddens 48, 0.00005 800
    for p2 in [0.00003]: # learning_rate
        for p3 in [900]: # subsample, max_iter

            avg_f1 = 0
            avg_m1 = 0
            avg_m2 = 0
            np.random.seed(113)
            sampler = SMOTE(sampling_strategy=0.5, random_state=113)

            print()
            print(p1, p2, p3)

            X = data_A[:,1].copy()
            Y = np.load("A_Y.npy")
            skf = StratifiedKFold(n_splits=5, random_state=113, shuffle=False)

            for kf, (id_train, id_test) in enumerate(skf.split(X, Y)):
                print("Fold %d/5" % (kf+1))

                X_all = vectorize(data_A[:,1].copy(), pre=True)
                Y = np.load("A_Y.npy")

                X_train = vectorize(data_A[id_train,1].copy(), pre=True)
                Y_train = np.load("A_Y.npy")[id_train]

                X_test = vectorize(data_A[id_test,1].copy(), pre=True)
                Y_test = np.load("A_Y.npy")[id_test]

                X_test = np.c_[X_test, find_len(data_A[id_test,1], tipe='A')]
                X_train = np.c_[X_train, find_len(data_A[id_train,1], tipe='A')]
                X_all = np.c_[X_all, find_len(data_A[:,1], tipe='A')]

                X_train, Y_train = sampler.fit_resample(X_train, Y_train)

                # # BASELINE
                clf_a_1 = XGBClassifier(random_state=113, n_estimators=75, learning_rate=0.001, subsample=0.75)
                clf_a_1.fit(X_train, Y_train)

                # MLP / 14
                clf_a_2 = MLPClassifier(random_state=113, max_iter=800, hidden_layer_sizes=(250,), learning_rate_init=0.0001, solver='adam')
                clf_a_2.fit(X_train, Y_train)

                A1 = np.log(clf_a_1.predict_proba(X_train))
                A2 = np.log(clf_a_2.predict_proba(X_train))
                out_A = np.c_[A1, A2]

                # r1 = 0.05
                # r2 = 0.35
                # r3 = 0.25
                # r4 = 0.35
                # tot_B = B1*r1 + B2*r2 + B3*r3 + B4*r4

                A1t = np.log(clf_a_1.predict_proba(X_test))
                A2t = np.log(clf_a_2.predict_proba(X_test))
                out_At = np.c_[A1t, A2t]
                # tot_Bt = B1t*r1 + B2t*r2 + B3t*r3 + B4t*r4

                ev1 = clf_a_1.predict(X_test)
                ev2 = clf_a_2.predict(X_test)

                print("==========================")
                print("M1 Accuracy pada test data:", accuracy_score(ev1, Y_test))
                print("   F1 pada test data:", f1_score(ev1, Y_test))
                print("M2 Accuracy pada test data:", accuracy_score(ev2, Y_test))
                print("   F1 pada test data:", f1_score(ev2, Y_test))
                print("==========================")

                meta_clf = MLPClassifier(
                        random_state=113,
                        max_iter=p3,
                        hidden_layer_sizes=p1,
                        learning_rate_init=p2,
                        learning_rate = 'adaptive',
                        solver='adam')

                meta_clf.fit(out_A, Y_train)
                prediction = meta_clf.predict(out_At).astype(int)
                # prediction = np.argmax(tot_Bt, 1)

                print("Accuracy pada test data:", accuracy_score(prediction, Y_test))
                print("F1 pada test data:", f1_score(prediction, Y_test))
                
                avg_f1 += f1_score(prediction, Y_test)
                avg_m1 += f1_score(ev1, Y_test)
                avg_m2 += f1_score(ev2, Y_test)
                            
            print("\nAverage F1", avg_f1/5)
            print("Average Lv0 F1-1", avg_m1/5)
            print("Average Lv0 F1-2", avg_m2/5)

            clf_a_1 = XGBClassifier(random_state=113, n_estimators=75, learning_rate=0.001, subsample=0.75)
            clf_a_2 = MLPClassifier(random_state=113, max_iter=800, hidden_layer_sizes=(250,), learning_rate_init=0.0001, solver='adam')
            clf_a_1.fit(X_all, Y)
            clf_a_2.fit(X_all, Y)
            A1 = np.log(clf_a_1.predict_proba(X_all))
            A2 = np.log(clf_a_2.predict_proba(X_all))
            out_A_all = np.c_[A1, A2]
            meta_clf = MLPClassifier(
                        random_state=113,
                        max_iter=p3,
                        hidden_layer_sizes=p1,
                        learning_rate_init=p2,
                        solver='adam')
            meta_clf.fit(out_A_all, Y)


(45,) 3e-05 900
Fold 1/5


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M1 Accuracy pada test data: 0.8363636363636363
   F1 pada test data: 0.888888888888889
M2 Accuracy pada test data: 0.8727272727272727
   F1 pada test data: 0.9135802469135802
Accuracy pada test data: 0.8727272727272727
F1 pada test data: 0.9135802469135802
Fold 2/5


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M1 Accuracy pada test data: 0.7592592592592593
   F1 pada test data: 0.8470588235294116
M2 Accuracy pada test data: 0.7592592592592593
   F1 pada test data: 0.8354430379746836


...

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M1 Accuracy pada test data: 0.8301886792452831
   F1 pada test data: 0.8831168831168831
M2 Accuracy pada test data: 0.9245283018867925
   F1 pada test data: 0.9444444444444444


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (900) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy pada test data: 0.9433962264150944
F1 pada test data: 0.958904109589041

Average F1 0.8933927078111237
Average Lv0 F1-1 0.8715379368249673
Average Lv0 F1-2 0.8899130580616637


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### DATA B

In [0]:
# avg_f1 = 0
# X = vectorize2(data_B[:,1].copy(), pre=True)
# # XE = np.load("B_X.npy")
# # X = vectorize(data_B[:,1].copy())
# Y = np.load("B_Y.npy")[:]

# # X = np.c_[X, XE]
# X = np.c_[X, find_len(data_B[:,1], tipe='B')]

# skf = StratifiedKFold(n_splits=5, random_state=11, shuffle=True)

# # hiddens = [(100,), (200,), (300,), (400,), (80,80),(120,120),(120,80)]
# hiddens = [(200,100), (128,64), (300, 100)]
# lris = [0.001, 0.0001, 0.005, 0.0005]
# # solvers = ['adam', 'sgd']
# max_iters = [400, 500, 600]
# # lrs = ['adaptive', 'constant', 'invscaling']

# hiddens = [(128,64)]
# lris = [0.0001]
# solvers = ['adam']
# max_iters = [500]
# lrs = ['adaptive']

# # ssmpls = [0.7,  0.8]
# # nestrs = [50, 75, 100, 125, 150, 175]
# # lris = [0.001, 0.01, 0.002, 0.008, 0.005]
# # mds = [3]
# # cbt = [0.8, 0.9, 1]

# # ssmpls = [0.8]
# # nestrs = [200]
# # mds = [3]
# # lris = [0.01]
# # cbt = [1]

# # for cb in cbt:
# #     for lri in lris:
# #         for ss in ssmpls:
# #             for ne in nestrs:
# #                 for md in mds:
# for h in hiddens:
#     for lri in lris:
#         for solver in solvers:
#             for mi in max_iters:
#                 for lr in lrs:
# # for h in range(1):
# #     for lri in range(1):
# #         for c in [0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1.0]:
# #             for gm in [0.01, 0.05, 0.1, 0.5, 1.0]:
# #                 for lr in range(1):
#                     print()
#                     print(h, lri, solver, mi, lr)
# #                     print(lri, ss, ne, md, cb)
# #                     print(c, gm)
                    
#                     avg_f1 = 0
            
#                     # BASELINE
# #                     clf2 = XGBClassifier(random_state=113, n_estimators=200, max_depth=3, learning_rate=0.01, subsample=0.8)

#                     # ATTEMPT 3
# #                     clf2 = AdaBoostClassifier(random_state=113, n_estimators=50, learning_rate=0.05)
    
#                     # ATTEMPT 5
# #                     clf2 = XGBClassifier(random_state=113, n_estimators=150, gamma=5, max_depth=3, learning_rate=0.01, subsample=0.8)
                    
    
#                     # ATTEMPT 12
# #                     clf2 = XGBClassifier(random_state=113, n_estimators=150, max_depth=3, learning_rate=0.0017, subsample=0.7, colsample_bytree=0.8)
            
#                     # Final attempt
#                     clf2 = MLPClassifier(
#                             random_state=113,
#                             max_iter=mi,
#                             hidden_layer_sizes=h,
#                             learning_rate_init=lri,
#                             learning_rate = lr,
#                             solver=solver)
    
# #                     clf2 = SVC(C=c, gamma=gm, probability=True, class_weight=None, random_state=113)
                    
#                     done = True
#                     for i, (train_index, test_index) in enumerate(skf.split(X, Y)):
#                         done = False
#                         x_train, y_train = X[train_index], Y[train_index]
# #                         x_train, y_train = sampler.fit_resample(x_train, y_train)

#                         x_test, y_test = X[test_index], Y[test_index]


#                         clf2.fit(x_train, y_train)

#                         prediction = clf2.predict(x_test).astype(int)

#                         from sklearn.metrics import accuracy_score
#                         from sklearn.metrics import precision_score
#                         from sklearn.metrics import recall_score
#                         from sklearn.metrics import f1_score

# #                         print("Accuracy pada train data:", accuracy_score(clf2.predict(x_train).astype(int), y_train))
# #                         print("Accuracy pada test data:", accuracy_score(prediction, y_test))
# #                         print("Precision baseline pada test data:", precision_score(prediction, y_test))
# #                         print("Recall baseline pada test data:", recall_score(prediction, y_test))
#                         print("F1 baseline pada test data:", f1_score(prediction, y_test))
# #                         print()
#                         avg_f1 += f1_score(prediction, y_test)

# #                         if f1_score(prediction, y_test)<0.64 and i==2:
# #                             break
# #                         if f1_score(prediction, y_test)<0.84 and i==3:
# #                             break
#                         done = True
                    
#                     if done:
#                         print("Average F1", avg_f1/5)

#                     # FINAL
#                     clf2 = MLPClassifier(
#                             random_state=113,
#                             max_iter=mi,
#                             hidden_layer_sizes=h,
#                             learning_rate_init=lri,
#                             learning_rate = lr,
#                             solver=solver)

#                     # BASELINE
# #                     clf2 = XGBClassifier(random_state=113, n_estimators=200, max_depth=3, learning_rate=0.01, subsample=0.8)
                    
#                     # 12
#                     # clf2 = XGBClassifier(random_state=113, n_estimators=150, max_depth=3, learning_rate=0.0017, subsample=0.7, colsample_bytree=0.8)
                    
#                     clf2.fit(X, Y)


In [0]:
# STACKING LV 0
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

avg_f1 = 0

# ACTIVATE FOR EXPLORING
for p1 in [70]: # n_estimator, hiddens
    for p2 in [0.02]: # learning_rate
        for p3 in [0.9]: # subsample, max_iter
# for p1 in [(128, 64), (500,), (600,)]: # n_estimator, hiddens
#     for p2 in [0.0001, 0.001]: # learning_rate
#         for p3 in [500, 300, 800]: # subsample, max_iter
            avg_exp_f1 = 0
            print()
            print(p1, p2, p3)

            np.random.seed(113)

            X = data_B[:,1].copy()
            XE = np.load("B_X.npy")
            Y = np.load("B_Y.npy")
#             X = np.c_[X, XE]
            skf = StratifiedKFold(n_splits=5, random_state=11, shuffle=True)

            for kf, (id_train, id_test) in enumerate(skf.split(X, Y)):
                print("Fold %d/5" % (kf+1))

                X_train = vectorize2(data_B[id_train,1].copy(), pre=True, tipe="B")
#                 X_train = XE[id_train]
                Y_train = np.load("B_Y.npy")[id_train]

                X_test = vectorize2(data_B[id_test,1].copy(), pre=True, tipe="B")
#                 X_test = XE[id_test]
                Y_test = np.load("B_Y.npy")[id_test]

                X_test = np.c_[X_test, XE[id_test], find_len(data_B[id_test,1], tipe='B')]
                X_train = np.c_[X_train, XE[id_train],find_len(data_B[id_train,1], tipe='B')]

                # # XGB
                clf_b_1 = XGBClassifier(random_state=113, n_estimators=p1,
                                        learning_rate=p2, subsample=p3,
                                        gamma=0, colsample_bytree=0.5)
                clf_b_1.fit(X_train, Y_train)

#                 # # MLP
#                 clf_b_2 = MLPClassifier(random_state=113, max_iter=p3, hidden_layer_sizes=p1, learning_rate_init=p2, solver='adam')
#                 clf_b_2.fit(X_train, Y_train)

                ev1 = clf_b_1.predict(X_test)
#                 ev2 = clf_b_2.predict(X_test)

                avg_exp_f1 += f1_score(ev1, Y_test) # <<<< change this

                print("M1 Accuracy pada test data:", accuracy_score(ev1, Y_test))
                print("   F1 pada test data:", f1_score(ev1, Y_test))
#                 print("M2 Accuracy pada test data:", accuracy_score(ev2, Y_test))
#                 print("   F1 pada test data:", f1_score(ev2, Y_test))

            print("AVERAGE F1", avg_exp_f1/5)


70 0.02 0.9
Fold 1/5
M1 Accuracy pada test data: 0.6290322580645161
   F1 pada test data: 0.7012987012987012
Fold 2/5
M1 Accuracy pada test data: 0.5806451612903226
   F1 pada test data: 0.675
Fold 3/5
M1 Accuracy pada test data: 0.7213114754098361
   F1 pada test data: 0.767123287671233
Fold 4/5
M1 Accuracy pada test data: 0.7
   F1 pada test data: 0.7428571428571429
Fold 5/5
M1 Accuracy pada test data: 0.7833333333333333
   F1 pada test data: 0.7936507936507938
AVERAGE F1 0.7359859850955741


In [0]:
# STACKING LV 1 FILM B
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

for p1 in [(32,)]: # n_estimator, hiddens
    for p2 in [0.00003]: # learning_rate
        for p3 in [1000]: # subsample, max_iter

            avg_f1 = 0
            avg_m1 = 0
            avg_m2 = 0
            avg_m3 = 0
            avg_m4 = 0
            np.random.seed(113)
            print()
            print(p1, p2, p3)

            X = data_B[:,1].copy()
            Y = np.load("B_Y.npy")
            skf = StratifiedKFold(n_splits=5, random_state=11, shuffle=True)

            for kf, (id_train, id_test) in enumerate(skf.split(X, Y)):
                print("Fold %d/5" % (kf+1))

                # id_test = np.random.choice(len(data_B), 39)
                # id_train = diff(range(len(data_B)), id_test)

                X_all = vectorize2(data_B[:,1].copy(), pre=True)
                Y = np.load("B_Y.npy")

                X_train = vectorize2(data_B[id_train,1].copy(), pre=True)
                Y_train = np.load("B_Y.npy")[id_train]

                X_test = vectorize2(data_B[id_test,1].copy(), pre=True)
                Y_test = np.load("B_Y.npy")[id_test]

                X_test = np.c_[X_test, find_len(data_B[id_test,1], tipe='B')]
                X_train = np.c_[X_train, find_len(data_B[id_train,1], tipe='B')]
                X_all = np.c_[X_all, find_len(data_B[:,1], tipe='B')]

                # # BASELINE
                # 12
                clf_b_2 = XGBClassifier(random_state=113, n_estimators=100, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8)
                clf_b_2.fit(X_train, Y_train)

                # MLP / 14
                clf_b_3 = MLPClassifier(random_state=113, max_iter=500, hidden_layer_sizes=(128,64), learning_rate_init=0.0001, solver='adam')
                clf_b_3.fit(X_train, Y_train)

                B2 = np.log(clf_b_2.predict_proba(X_train))
                B3 = np.log(clf_b_3.predict_proba(X_train))
                out_B = np.c_[B2, B3]

                B2t = np.log(clf_b_2.predict_proba(X_test))
                B3t = np.log(clf_b_3.predict_proba(X_test))
                out_Bt = np.c_[B2t, B3t]

                ev2 = clf_b_2.predict(X_test)
                ev3 = clf_b_3.predict(X_test)

                print("==========================")
                print("M2 Accuracy pada test data:", accuracy_score(ev2, Y_test))
                print("   F1 pada test data:", f1_score(ev2, Y_test))
                print("M3 Accuracy pada test data:", accuracy_score(ev3, Y_test))
                print("   F1 pada test data:", f1_score(ev3, Y_test))
                print("==========================")

                meta_clf2 = MLPClassifier(
                        random_state=113,
                        max_iter=p3,
                        hidden_layer_sizes=p1,
                        learning_rate_init=p2,
                        learning_rate = 'adaptive',
                        solver='adam')

                meta_clf2.fit(out_B, Y_train)

                prediction = meta_clf2.predict(out_Bt).astype(int)

                print("Accuracy pada test data:", accuracy_score(prediction, Y_test))
                print("F1 pada test data:", f1_score(prediction, Y_test))
                
                avg_f1 += f1_score(prediction, Y_test)
                avg_m2 += f1_score(ev2, Y_test)
                avg_m3 += f1_score(ev3, Y_test)

                            
            print("\nAverage F1", avg_f1/5)
            print("Average Lv0 F1-2", avg_m2/5)
            print("Average Lv0 F1-3", avg_m3/5)

            clf_b_2 = XGBClassifier(random_state=113, n_estimators=100, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8)
            clf_b_3 = MLPClassifier(random_state=113, max_iter=500, hidden_layer_sizes=(128,64), learning_rate_init=0.0001, solver='adam')
            clf_b_2.fit(X_all, Y)
            clf_b_3.fit(X_all, Y)
            B2 = np.log(clf_b_2.predict_proba(X_all))
            B3 = np.log(clf_b_3.predict_proba(X_all))
            out_B_all = np.c_[B2, B3]
            meta_clf2 = MLPClassifier(
                        random_state=113,
                        max_iter=p3,
                        hidden_layer_sizes=p1,
                        learning_rate_init=p2,
                        solver='adam')
            meta_clf2.fit(out_B_all, Y)


(32,) 3e-05 1000
Fold 1/5


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M2 Accuracy pada test data: 0.6290322580645161
   F1 pada test data: 0.6666666666666666
M3 Accuracy pada test data: 0.6612903225806451
   F1 pada test data: 0.6956521739130436


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy pada test data: 0.6612903225806451
F1 pada test data: 0.7042253521126761
Fold 2/5


...

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


M2 Accuracy pada test data: 0.6833333333333333
   F1 pada test data: 0.7164179104477613
M3 Accuracy pada test data: 0.7666666666666667
   F1 pada test data: 0.7741935483870968


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy pada test data: 0.75
F1 pada test data: 0.7761194029850745

Average F1 0.7607485848868271
Average Lv0 F1-2 0.7441397949014403
Average Lv0 F1-3 0.7559920031396032


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Classifier LSTM

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

def seed():
    torch.manual_seed(113)
    torch.cuda.manual_seed(113)
    torch.cuda.manual_seed_all(113)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    np.random.seed(113)

    import random
    random.seed(113)

def _init_fn(worker_id):
    np.random.seed(int(seed))
    
seed()

In [0]:
len_seq = 14
def batcher(sent):
    ret = []
    counter = 0
    null = np.zeros(150)
    for word in sent.split():
        if word in ["./,;!?()"]:
            continue
        word = word.lower()
        ret.append(model[word])
    while(len(ret) < len_seq):
        ret.insert(0, null)
    while(len(ret) > len_seq):
        ret.pop(-1)  
    return np.asarray(ret)

def batcher2(sent):
    ret = []
    counter = 0
    null = np.zeros(300)
    for word in sent.split():
        if word in ["./,;!?()"]:
            continue
        ret.append(model2[word])
    while(len(ret) < len_seq):
        ret.insert(0, null)
    while(len(ret) > len_seq):
        ret.pop(-1)  
    return np.asarray(ret)

def data_batcher(x):
    res = []
    for i in x:
        res.append(batcher(i[1]))
    return np.asarray(res) 

In [0]:
# xwer_A = data_batcher(xtrain_A)
# xwer_B = data_batcher(xtrain_B)
# xwet_A = data_batcher(xtest_A)
# xwet_B = data_batcher(xtest_B)

# yr_A = np.asarray(ytrain_A).astype(int)
# yt_A = np.asarray(ytest_A).astype(int)
# yr_B = np.asarray(ytrain_B).astype(int)
# yt_B = np.asarray(ytest_A).astype(int)

In [0]:
class LSTMModel(nn.Module):   

    def __init__(self, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(LSTMModel, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.5)
        
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        
        batch_size = x.size(0)
        seq_len = x.size(1)

        lstm_out, hidden = self.lstm(x, hidden)
#         print(lstm_out.shape)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
#         print(lstm_out.shape)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = F.relu(out)
        
        out = out.view(batch_size, seq_len, -1)
#         print(out.shape)
        out = out[:, -1]
#         print(out.shape)
        return F.log_softmax(out), hidden
    
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        
        return hidden

In [0]:

# def reset_data():
# dataloaders
batch_size = 42

# make sure the SHUFFLE your training data
train_loader = []
valid_loader = []
test_size = [0.2, 0.15, 0.1, 0.2, 0.15]
rand_i = [113, 1117, 11, 3, 317]

for i in range(5):
    xtrain_A, xtest_A, ytrain_A, ytest_A = train_test_split(data_A[:,:-1], data_A[:,-1],
                                      test_size=test_size[i], 
                                      stratify=data_A[:,-1], 
                                      random_state=rand_i[i])

    xtrain_B, xtest_B, ytrain_B, ytest_B = train_test_split(data_B[:,:-1], data_B[:,-1],
                                          test_size=test_size[i], 
                                          stratify=data_B[:,-1], 
                                          random_state=rand_i[i])

    xwer_A = data_batcher(xtrain_A)
    xwer_B = data_batcher(xtrain_B)
    xwet_A = data_batcher(xtest_A)
    xwet_B = data_batcher(xtest_B)

    yr_A = np.asarray(ytrain_A).astype(int)
    yt_A = np.asarray(ytest_A).astype(int)
    yr_B = np.asarray(ytrain_B).astype(int)
    yt_B = np.asarray(ytest_A).astype(int)

    train_data = TensorDataset(torch.from_numpy(xwer_A), torch.from_numpy(yr_A))
    valid_data = TensorDataset(torch.from_numpy(xwet_A), torch.from_numpy(yt_A))

    train_loader.append(DataLoader(train_data, shuffle=True, batch_size=batch_size, worker_init_fn=_init_fn))
    valid_loader.append(DataLoader(valid_data, shuffle=True, batch_size=batch_size, worker_init_fn=_init_fn))
        
# reset_data()

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

seed()

output_size = 2
embedding_dim = 150
hidden_dim = 256
n_layers = 1
lrs= 0.00007
epochs = 45
counter = 0
print_every = 500
clip=5 # gradient clipping

# for epoch in epochs:
#     for lr in lrs:
#         for hidden_dim in hidden_dims:
#             for n_layer in n_layers:
                
#                 print()
#                 print(epoch, lr, hidden_dim, n_layer)

avg_f1 = 0

for i in range(5):

    clf = LSTMModel(output_size, embedding_dim, hidden_dim, n_layers)
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(clf.parameters(), lr=lrs)

    clf.cuda()
    clf.train()

    for e in range(epochs):

        for _, (inputs, labels) in enumerate(train_loader[i]):

            counter += 1

            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.float()

            h = clf.init_hidden(inputs.size(0))
            h = tuple([each.data for each in h])

            clf.zero_grad()
            output, h = clf(inputs, h)

            loss = criterion(output.squeeze(), labels)
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
#             nn.utils.clip_grad_norm_(clf.parameters(), clip)
            optimizer.step()

        clf.eval()
        val_losses = []
        acc = torch.Tensor([]).cuda()
        for inputs, labels in valid_loader[i]:

            val_h = clf.init_hidden(inputs.size(0))
            val_h = tuple([each.data for each in val_h])

            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.float()

            output, val_h = clf(inputs, val_h)
    #         print(labels)
    #         print(torch.argmax(output.squeeze(), dim=1))
            pred = torch.argmax(output.squeeze(), dim=1)==labels
            acc = torch.cat((acc, pred.float()),0)
            val_loss = criterion(output.squeeze(), labels)

            val_losses.append(val_loss.item())

        clf.train()
#                         if e%print_every == 0:
#                             print("Epoch: {}/{}...".format(e+1, epochs),
#                                   "Loss: {:.6f}...".format(loss.item()),
#                                   "Val Loss: {:.6f}".format(np.mean(val_losses)),
#                                   "Val Acc: {:.6f}".format(torch.mean(acc)))


    clf.eval()
    preds = torch.Tensor([]).cuda()
    labels_all = torch.Tensor([]).cuda()
    for inputs, labels in valid_loader[i]:

        val_h = clf.init_hidden(inputs.size(0))
        val_h = tuple([each.data for each in val_h])

        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = inputs.float()

        output, val_h = clf(inputs, val_h)
    #     print(output.shape)
        pred = torch.argmax(output.squeeze(), dim=1)
        preds = torch.cat((preds, pred.float()),0)
        labels_all = torch.cat((labels_all, labels.float()),0)

    preds = preds.detach().cpu().numpy()
    labels_all = labels_all.cpu().numpy()
#                     print(preds)

    print("Accuracy pada test data:", accuracy_score(preds, labels_all))
    print("Accuracy pada test data:", accuracy_score(preds, labels_all))
    print("Precision baseline pada test data:", precision_score(preds, labels_all))
    print("Recall baseline pada test data:", recall_score(preds, labels_all))
    print("F1 baseline pada test data:", f1_score(preds, labels_all))
    print()
    avg_f1 += f1_score(preds, labels_all)


print(avg_f1/5)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy pada test data: 0.7777777777777778
Precision baseline pada test data: 0.868421052631579
Recall baseline pada test data: 0.825
F1 baseline pada test data: 0.8461538461538461

Accuracy pada test data: 0.8048780487804879
Precision baseline pada test data: 0.9655172413793104
Recall baseline pada test data: 0.8
F1 baseline pada test data: 0.8750000000000001

Accuracy pada test data: 0.7777777777777778
Precision baseline pada test data: 0.8421052631578947
Recall baseline pada test data: 0.8421052631578947
F1 baseline pada test data: 0.8421052631578947

Accuracy pada test data: 0.7592592592592593
Precision baseline pada test data: 0.9473684210526315
Recall baseline pada test data: 0.7659574468085106
F1 baseline pada test data: 0.8470588235294116

Accuracy pada test data: 0.8292682926829268
Precision baseline pada test data: 0.9655172413793104
Recall baseline pada test data: 0.8235294117647058
F1 baseline pada test data: 0.888888888888889

0.8598413643460084


In [0]:
# dataloaders
batch_size = 42

# make sure the SHUFFLE your training data
train_loader = []
valid_loader = []
test_size = [0.2, 0.15, 0.1, 0.2, 0.15]
rand_i = [113, 1117, 11, 3, 317]

for i in range(5):
    xtrain_B, xtest_B, ytrain_B, ytest_B = train_test_split(data_B[:,:-1], data_B[:,-1],
                                          test_size=test_size[i], 
                                          stratify=data_B[:,-1], 
                                          random_state=rand_i[i])
    
    xwer_B = data_batcher(xtrain_B)
    xwet_B = data_batcher(xtest_B)

    yr_B = np.asarray(ytrain_B).astype(int)
    yt_B = np.asarray(ytest_A).astype(int)
    
    train_data = TensorDataset(torch.from_numpy(xwer_B), torch.from_numpy(yr_B))
    valid_data = TensorDataset(torch.from_numpy(xwet_B), torch.from_numpy(yt_B))
    
    train_loader.append(DataLoader(train_data, shuffle=True, batch_size=batch_size, worker_init_fn=_init_fn))
    valid_loader.append(DataLoader(valid_data, shuffle=True, batch_size=batch_size, worker_init_fn=_init_fn))

In [0]:
seed()

output_size = 2
embedding_dim = 300
hidden_dim = 500
n_layers = 1
lr=0.00007
epochs = 75 # 3-4 is approx where I noticed the validation loss stop decreasing
counter = 0
print_every = 30
clip=5 # gradient clipping

avg_f1 = 0

for i in range(5):
    
    clf = LSTMModel(output_size, embedding_dim, hidden_dim, n_layers)
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(clf.parameters(), lr=lr)
    
    clf.cuda()
    clf.train()
    
    for e in range(epochs):

        for inputs, labels in train_loader[i]:

            counter += 1

            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.float()

            h = clf.init_hidden(inputs.size(0))
            h = tuple([each.data for each in h])

            clf.zero_grad()
            output, h = clf(inputs, h)

            loss = criterion(output.squeeze(), labels)
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(clf.parameters(), clip)
            optimizer.step()

        clf.eval()
        val_losses = []
        acc = torch.Tensor([]).cuda()
        for inputs, labels in valid_loader[i]:

            val_h = clf.init_hidden(inputs.size(0))
            val_h = tuple([each.data for each in val_h])

            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.float()

            output, val_h = clf(inputs, val_h)
    #         print(labels)
    #         print(torch.argmax(output.squeeze(), dim=1))
            pred = torch.argmax(output.squeeze(), dim=1)==labels
            acc = torch.cat((acc, pred.float()),0)
            val_loss = criterion(output.squeeze(), labels)

            val_losses.append(val_loss.item())

        clf.train()
        if e%print_every == 0:
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Acc: {:.6f}".format(torch.mean(acc)))


    clf.eval()
    preds = torch.Tensor([]).cuda()
    labels_all = torch.Tensor([]).cuda()
    for inputs, labels in valid_loader[i]:

        val_h = clf.init_hidden(inputs.size(0))
        val_h = tuple([each.data for each in val_h])

        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = inputs.float()

        output, val_h = clf(inputs, val_h)
    #     print(output.shape)
        pred = torch.argmax(output.squeeze(), dim=1)
        preds = torch.cat((preds, pred.float()),0)
        labels_all = torch.cat((labels_all, labels.float()),0)

    preds = preds.detach().cpu().numpy()
    labels_all = labels_all.cpu().numpy()
    print(preds)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score

    print("Accuracy pada test data:", accuracy_score(preds, labels_all))
    print("Precision baseline pada test data:", precision_score(preds, labels_all))
    print("Recall baseline pada test data:", recall_score(preds, labels_all))
    print("F1 baseline pada test data:", f1_score(preds, labels_all))
    print()
    avg_f1 += f1_score(preds, labels_all)
    
    
print(avg_f1/5)

# Submission

In [0]:
data_dev_A_df = pd.read_csv("drive/My Drive/Ukara/data_dev_A.csv")
data_dev_B_df = pd.read_csv("drive/My Drive/Ukara/data_dev_B.csv")
data_dev_A = data_dev_A_df.values
data_dev_B = data_dev_B_df.values

In [0]:
data_tes_A_df = pd.read_csv("drive/My Drive/Ukara/data_test_A.csv")
data_tes_B_df = pd.read_csv("drive/My Drive/Ukara/data_test_B.csv")
data_tes_A = data_tes_A_df.values
data_tes_B = data_tes_B_df.values

# ppmi_test_a = pd.read_csv(path_ukara + 'data_test_A_ppmi.csv')['PPMI'].values

In [0]:
data_tes_A[0]

array(['TSA1',
       'beradaptasi dengan lingkunagn baru, lahan dan harta mereka yang dulu akan hilang, tertinggalnya teknologi karena bencana lingkunagn tersebut.'],
      dtype=object)

In [0]:
# DEV
# xfd_A = vectorize(data_dev_A[:,1], pre=True)
# xfd_B = vectorize2(data_dev_B[:,1], pre=True)

# xfd_A = np.c_[xfd_A, find_len(data_dev_A[:,1], tipe='A'), ppmi_test_a]
# xfd_B = np.c_[xfd_B, find_len(data_dev_B[:,1], tipe='B')]

# predict_A = clf.predict(xfd_A)
# predict_B = clf2.predict(xfd_B)

In [0]:
xft_A = vectorize(data_tes_A[:,1], pre=True)
xft_B = vectorize2(data_tes_B[:,1], pre=True)

# STACKING DATA
xft_A = np.c_[xft_A, find_len(data_tes_A[:,1], tipe='A')]
xft_B = np.c_[xft_B, find_len(data_tes_B[:,1], tipe='B')]

A1 = np.log(clf_a_1.predict_proba(xft_A))
A2 = np.log(clf_a_2.predict_proba(xft_A))
B1 = np.log(clf_b_2.predict_proba(xft_B))
B2 = np.log(clf_b_3.predict_proba(xft_B))
# r1 = 0.05
# r2 = 0.35
# r3 = 0.25
# r4 = 0.35
# tot_B = B1*r1 + B2*r2 + B3*r3 + B4*r4

out_A = np.c_[A1, A2]
out_B = np.c_[B1, B2]

predict_tes_A = meta_clf.predict(out_A)
predict_tes_B = meta_clf2.predict(out_B)
# predict_tes_B = np.argmax(tot_B, 1)

In [0]:
# # PREDICT NON_STACKING
xft_A = vectorize(data_tes_A[:,1].copy(), pre=True)
xft_B = vectorize2(data_tes_B[:,1].copy(), pre=True)
xft_A = np.c_[xft_A, find_len(data_tes_A[:,1], tipe='A'), ppmi_test_a]
xft_B = np.c_[xft_B, find_len(data_tes_B[:,1], tipe='B')]

predict_tes_A = clf.predict(xft_A)
predict_tes_B = clf2.predict(xft_B)

In [0]:
for i in range(len(data_tes_A)):
    if len(str(data_tes_A[i,1]))<=5:
        print(data_tes_A[i,0], predict_tes_A[i])
        predict_tes_A[i]=0

for i in range(len(data_tes_B)):
    if len(str(data_tes_B[i,1]))<=5:
        print(data_tes_B[i,0], predict_tes_B[i])
        predict_tes_B[i]=0

TSA369 0
TSA373 0
TSA390 0
TSA465 0
TSA536 1
TSA715 0
TSB592 0
TSB751 0


In [0]:
# from pandas import DataFrame

# prediksi_data_A = {'RES_ID': data_dev_A_df['RES_ID'],
#                 'LABEL': predict_A
#                 }

# prediksi_data_B = {'RES_ID': data_dev_B_df['RES_ID'],
#                 'LABEL': predict_B
#                 }

In [0]:
# df_data_A = DataFrame(prediksi_data_A, columns= ['RES_ID', 'LABEL'])
# df_data_B = DataFrame(prediksi_data_B, columns= ['RES_ID', 'LABEL'])
# frame = [df_data_A, df_data_B]
# answer = pd.concat(frame)

In [0]:
# answer.to_json('ANS_BEST_DEV.json', orient='records')

In [0]:
from pandas import DataFrame

prediksi_data_A = {'RES_ID': data_tes_A_df['RES_ID'],
                'LABEL': predict_tes_A
                }

prediksi_data_B = {'RES_ID': data_tes_B_df['RES_ID'],
                'LABEL': predict_tes_B
                }

df_data_A = DataFrame(prediksi_data_A, columns= ['RES_ID', 'LABEL'])
df_data_B = DataFrame(prediksi_data_B, columns= ['RES_ID', 'LABEL'])
frame = [df_data_A, df_data_B]
answer = pd.concat(frame)

answer.to_json('ANS_STACKING_ANEWPRE_BFINAL.json', orient='records')

# TRIAL SECTION